In [ ]:
import pandas as pd 
import numpy as np
import glob
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.models import resnet18

from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

## Data set

Before we deal with the models, we need to understand how to upload the data set. Let's write a torch class for this.

In [ ]:
class FaceDataset(Dataset):
    def __init__(self, data_df, transform=None):
        """
        Now we initialize the class by sending it a table 
        with image data 
        """
        self.data_df = data_df
        self.transform = transform

    def __getitem__(self, idx):
        # we get the name of the image and its label
        image_name, label = self.data_df.iloc[idx]['name'], self.data_df.iloc[idx]['label']
        
        # read the image
        image = cv2.imread(f"../input/data256faces/data256/train/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        
        # transform it, if necessary
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label).long()
    
    def __len__(self):
        return len(self.data_df)

In [ ]:
we set the image transformation 

train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

valid_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

In [ ]:
# read the data set

data_df = pd.read_csv("../input/train.csv")

In [ ]:
# let's look at the images

fig, axs = plt.subplots(2, 4, figsize=(16,8))
fig.suptitle(f'True images {" "*105} Fake images', fontsize=14)

for i, name in zip(range(4), data_df[ data_df['label'] == 0 ].sample(4, random_state=42)['name']):
    axs[i // 2, (i % 2)].imshow(plt.imread(f"../input/data256faces/data256/train/{name}"))
    axs[i // 2, (i % 2)].axis('off')

for i, name in zip(range(4), data_df[ data_df['label'] == 1 ].sample(4, random_state=42)['name']):
    axs[i // 2, (i % 2)+2].imshow(plt.imread(f"../input/data256faces/data256/train/{name}"))
    axs[i // 2, (i % 2)+2].axis('off')

fig.tight_layout()
fig.subplots_adjust(top=0.88)

They look quite similar, right?

In [ ]:
# let's divide the data set into train and validation to check the quality 

train_df, valid_df = train_test_split(data_df, test_size=0.3, random_state=42)

In [ ]:
train_df.shape, valid_df.shape

In [ ]:
train_dataset = FaceDataset(train_df, train_transform)
valid_dataset = FaceDataset(valid_df, valid_transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=2)

valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                           batch_size=64,
                                           # shuffle=True,
                                           pin_memory=True,
                                           num_workers=2)

Great, now let's write a couple of helper functions for visualization and training.

## Helper functions

In [ ]:
def plot_history(train_history, val_history, title='loss'):
    plt.figure()
    plt.title('{}'.format(title))
    plt.plot(train_history, label='train', zorder=1)
    
    points = np.array(val_history)
    steps = list(range(0, len(train_history) + 1, int(len(train_history) / len(val_history))))[1:]
    
    plt.scatter(steps, val_history, marker='+', s=180, c='orange', label='val', zorder=2)
    plt.xlabel('train steps')
    
    plt.legend(loc='best')
    plt.grid()

    plt.show()

In [ ]:
def train(model, criterion, optimizer, train_dataloader, test_dataloader, NUM_EPOCH=15):
    train_loss_log = []
    val_loss_log = []
    
    train_acc_log = []
    val_acc_log = []
    
    for epoch in tqdm(range(NUM_EPOCH)):
        print(epoch)
        model.train()
        train_loss = 0.
        train_size = 0
        
        train_pred = 0.

        for imgs, labels in train_dataloader:
            optimizer.zero_grad()

            imgs = imgs.cuda()
            labels = labels.cuda()

            y_pred = model(imgs)

            loss = criterion(y_pred, labels)
            loss.backward()
            
            train_loss += loss.item()
            train_size += y_pred.size(0)
            train_loss_log.append(loss.data / y_pred.size(0))
            
            train_pred += (y_pred.argmax(1) == labels).sum()

            optimizer.step()
        
        train_acc_log.append(train_pred / train_size)

        val_loss = 0.
        val_size = 0
        
        val_pred = 0.
        
        model.eval()
        
        with torch.no_grad():
            for imgs, labels in test_dataloader:
                
                imgs = imgs.cuda()
                labels = labels.cuda()
                
                pred = model(imgs)
                loss = criterion(pred, labels)
                
                val_loss += loss.item()
                val_size += pred.size(0)
                
                val_pred += (pred.argmax(1) == labels).sum()

        val_loss_log.append(val_loss / val_size)
        val_acc_log.append(val_pred / val_size)

        clear_output()
        plot_history(train_loss_log, val_loss_log, 'loss')

        print('Train loss:', (train_loss / train_size)*100)
        print('Val loss:', (val_loss / val_size)*100)
        print('Train acc:', (train_pred / train_size)*100)
        print('Val acc:', (val_pred / val_size)*100)
        
    return train_loss_log, train_acc_log, val_loss_log, val_acc_log

## Model
That's it, let's move on to training the model itself. Let's use a pre-trained ResNet and replace its classifier, and then we will train only it.

In [ ]:
# We upload the model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 2)

model = model.cuda()
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.01)

In [ ]:
train_loss_log, train_acc_log, val_loss_log, val_acc_log = train(model, 
                                                                 criterion, 
                                                                 optimizer, 
                                                                 train_loader, 
                                                                 valid_loader, 
                                                                 15)

## Let's look at the metrics of our final model for validation

In [ ]:
model.eval()
valid_predicts = []

for imgs, _ in tqdm(valid_loader):
    
    imgs = imgs.cuda()
    pred = model(imgs)
    
    valid_predicts.append(pred.softmax(dim=1)[:, 1].cpu().detach().numpy()) # argmax(dim=1)

In [ ]:
valid_predicts = np.concatenate(valid_predicts)

In [ ]:
valid_df['pred'] = valid_predicts

Let's calculate accuracy.

In [ ]:
val_acc = (valid_df['label'].values == (valid_df['pred'].values >= 0.5)*1).mean()
print(f"Validation accuracy = {val_acc}")

And now roc-auc as well

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

In [ ]:
val_rocauc = roc_auc_score(valid_df['label'].values, valid_df['pred'].values)
print(f"Validation ROC-AUC = {val_rocauc}")

In [ ]:
fpr, tpr, threshold = roc_curve(y_true=valid_df['label'].values, 
                                        y_score=valid_df['pred'].values, 
                                        pos_label=1)
roc_auc = auc(fpr, tpr)
plt.title('ROC-AUC curve')
plt.plot(fpr, tpr, c='b', label='AUC = '+str(np.round(roc_auc,3)))
plt.xlabel('False PR', fontsize=15)
plt.ylabel('True PR', fontsize=15)
plt.legend()
plt.grid(True)
plt.show()

Looks good!

## Let's predict for the test

In [ ]:
test_paths = glob.glob("../input/data256faces/data256/test/*.png")

In [ ]:
test_names = [t_path.split('/')[-1] for t_path in test_paths]

In [ ]:
test_df = pd.DataFrame(test_names, columns=['name'])

In [ ]:
class TestFaceDataset(Dataset):
    def __init__(self, data_df, transform=None):
        self.data_df = data_df
        self.transform = transform

    def __getitem__(self, idx):
        image_name = self.data_df.iloc[idx]['name']
        
        image = cv2.imread(f"../input/data256faces/data256/test/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        
        if self.transform:
            image = self.transform(image)
        
        return image
    
    def __len__(self):
        return len(self.data_df)

In [ ]:
test_dataset = TestFaceDataset(test_df, valid_transform)

In [ ]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=64,
                                           # shuffle=True,
                                           pin_memory=True,
                                           num_workers=2)

We predict the result using our model

In [ ]:
model.eval()
predicts = []

for imgs in tqdm(test_loader):
    
    imgs = imgs.cuda()
    pred = model(imgs)
    
    predicts.append(pred.softmax(dim=1)[:, 1].cpu().detach().numpy()) # argmax(dim=1)

In [ ]:
predicts = np.concatenate(predicts)

In [ ]:
submit_df = pd.DataFrame([[name, pred] for name, pred in zip(test_names, predicts)], columns=['name', 'pred'])

In [ ]:
submit_df = submit_df.sort_values(['name'])
submit_df.sample(5)

We get the final file, 'submit.csv', sorted by name

In [ ]:
submit_df.to_csv("submit.csv", index=False)